In [1]:
import cv2
import numpy as np

In [2]:
#****************************************************************************

cam = cv2.VideoCapture(0,cv2.CAP_DSHOW)

def nothing(c):
    pass

# Creating a window with black image
# img = np.zeros((300, 512, 3), np.uint8)
cv2.namedWindow('Adjustment bar')

cv2.createTrackbar('Threshold', 'Adjustment bar', 0, 255, nothing)
cv2.createTrackbar('L_Hue', 'Adjustment bar', 0, 255, nothing)
cv2.createTrackbar('L_Sat', 'Adjustment bar', 0, 255, nothing)
cv2.createTrackbar('L_Bright', 'Adjustment bar', 0, 255, nothing)

cv2.createTrackbar('U_Hue', 'Adjustment bar', 255, 255, nothing)
cv2.createTrackbar('U_Sat', 'Adjustment bar', 255, 255, nothing)
cv2.createTrackbar('U_Bright', 'Adjustment bar', 255, 255, nothing)

while (True):
    _, frame1 = cam.read()
    
  
    cv2.rectangle(frame1, (30,30),(500,500),(255, 0, 0), 5)
    frame = frame1[40:420, 40:420]
    frame = cv2.resize(frame, (300,300))
    hsv = cv2.cvtColor(frame, cv2.COLOR_BGR2HSV)

# Detecting Hand
    L_hue = cv2.getTrackbarPos('L_Hue', 'Adjustment bar')
    L_sat = cv2.getTrackbarPos('L_Sat', 'Adjustment bar')
    L_Bright = cv2.getTrackbarPos('L_Bright', 'Adjustment bar')

    U_Hue = cv2.getTrackbarPos('U_Hue', 'Adjustment bar')
    U_Sat = cv2.getTrackbarPos('U_Sat', 'Adjustment bar')
    U_Bright = cv2.getTrackbarPos('U_Bright', 'Adjustment bar')

    lower_bound = np.array([L_hue, L_sat, L_Bright])
    upper_bound = np.array([U_Hue, U_Sat, U_Bright])
    
    # Create Mask
    
    mask = cv2.inRange(hsv, lower_bound, upper_bound)
    #filter mask with image
    filter = cv2.bitwise_and(frame, frame, mask=mask)
    
    # Final Mask - making background - Black and foreground - White
    mask1  = cv2.bitwise_not(mask)
    
    # Obtain Threshold value from trackbar
    m_g = cv2.getTrackbarPos("Threshold", "Adjustment bar")
    
    # create threshold
    
    ret,thresh = cv2.threshold(mask1,m_g,255,cv2.THRESH_BINARY)
        
    # Smoothening the mask
    
    dilate = cv2.dilate(thresh,(1,1),iterations = 3)
    
    ##-------------------------------------------------------------------------------------------##
    ##-------------------------------------------------------------------------------------------##
    ##-------------------------------------------------------------------------------------------##

    # Find Contours
    
    cnts,hier = cv2.findContours(thresh,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
    
    # Draw Counters
    
    contours = cv2.drawContours(mask1,cnts,-1,(176,10,15),5)
    contours1 = cv2.drawContours(frame,cnts,-1,(12,250,250),5)
    
        
    ##-------------------------------------------------------------------------------------------##
    ##-------------------------------------------------------------------------------------------##
    ##-------------------------------------------------------------------------------------------##

    # Convex Hull
    
    
    for c in cnts:
        epsilon = 0.0001*cv2.arcLength(c,True)
        data= cv2.approxPolyDP(c,epsilon,True)
    
        hull = cv2.convexHull(data)
        cv2.drawContours(frame, [c], -1, (50, 50, 150), 2)
        cv2.drawContours(frame, [hull], -1, (0, 255, 0), 2)
    
    
    
    
    cv2.imshow("asdd", frame1)
    cv2.imshow("Original", frame)
    cv2.imshow("Mask", mask)
    cv2.imshow("Filtered Mask", filter)
    cv2.imshow("Final Mask", mask1)
#     cv2.imshow("Threshold", thresh)




    key = cv2.waitKey(25) & 0xFF
    if key == 27:
        break
cam.release()        
cv2.destroyAllWindows()

TypeError: ord() expected string of length 1, but int found